In [32]:
import pandas as pd
import os

query_path = "/crawl/crawler/query_output"
query_path_list= []

for cat in os.listdir(query_path):
    if cat == 'general':
        continue
    else:
        sub_path = os.path.join(query_path,cat)

        for cat_2 in os.listdir(sub_path):
            edge_path = os.path.join(sub_path, cat_2)

            for cat_3 in os.listdir(edge_path):
                file_path = os.path.join(edge_path, cat_3)
                query_path_list.append(file_path)

### Results load analysis

In [33]:
# DRAFT
with open ("ss_10000_factsNet_finetuned_DRAFT_results.log", 'r') as f:
    DRAFT_ss_10000 = f.readlines()
    
with open ("ss_500_factsnet_DRAFT.log", 'r') as f:
    DRAFT_ss_500 = f.readlines()

# with open ("ss_50_factsNet_finetuned_DRAFT_results.log", 'r') as f:
#     DRAFT_ss_50 = f.readlines()

# gpt3 ada
with open ("0103_ORIGINAL_GPT3_ADA_factsNet_zero-shot_0-shot_results.log", "r") as f:
    gpt3_ada_zero_shot = f.readlines()

with open ("0103_ORIGINAL_GPT3_ADA_factsNet_few-shot_1-shot_results.log", "r") as f:
    gpt3_ada_one_shot = f.readlines()

with open ("0103_ORIGINAL_GPT3_ADA_factsNet_few-shot_3-shot_results.log", "r") as f:
    gpt3_ada_three_shot = f.readlines()

with open ("0103_ORIGINAL_GPT3_ADA_factsNet_few-shot_5-shot_results.log", "r") as f:
    gpt3_ada_five_shot = f.readlines()

# gpt3 davinci
with open ("0103_ORIGINAL_GPT3_DAVINCI_factsNet_zero-shot_0-shot_results.log", 'r') as f:
    gpt3_davinci_zero_shot = f.readlines()

with open ("0103_ORIGINAL_GPT3_DAVINCI_factsNet_few-shot_1-shot_results.log", "r") as f:
    gpt3_davinci_one_shot = f.readlines()

with open ("0103_ORIGINAL_GPT3_DAVINCI_factsNet_few-shot_3-shot_results.log", "r") as f:
    gpt3_davinci_three_shot = f.readlines()

with open ("0103_ORIGINAL_GPT3_DAVINCI_factsNet_few-shot_5-shot_results.log", 'r') as f:
    gpt3_davinci_five_shot = f.readlines()

# weak baselines
with open ("factsNet_baselines_random_results_NEW.log", 'r') as f:
    random = f.readlines()

with open ("factsNet_baselines_only_query_based_results_NEW.log", 'r') as f:
    only_query = f.readlines()

with open ("factsNet_baselines_token_level_freq_results_NEW.log", 'r') as f:
    tok_freq = f.readlines()


In [34]:
def make_results(data, line_width):
    
    results = pd.DataFrame(index=range(0,len(data) // line_width), columns=['content','f1','acc','rec','prec'])

    k = 0
    for i, res in enumerate(data):
        if i % line_width == 0:

            k = i // line_width
        res = res.replace('\n','')

        if 'content_name' in res:
            # results['content'][i] = res.split(': ')[1].replace('query_','')
            results.loc[k,'content'] = res.split(': ')[1].replace('query_','')
        elif 'f1-score' in res:
            # results['f1'][i] = float(res.split(': ')[1])
            results.loc[k,'f1']= float(res.split(': ')[1])
        elif 'accuracy' in res:
            # results['acc'][i] = float(res.split(': ')[1])
            results.loc[k,'acc']= float(res.split(': ')[1])
        elif 'recall' in res:
            # results['rec'][i] = float(res.split(': ')[1])
            results.loc[k,'rec']= float(res.split(': ')[1])
        elif 'precision' in res:
            # results['prec'][i] = float(res.split(': ')[1])
            results.loc[k,'prec']= float(res.split(': ')[1])
        else:
            continue
    
    # results = results.sort_values(by=['content'],ascending=True)
    return results


def analysis_df(results):
    results['large'] = 0 
    results['middle'] = 0

    for i in range(results.shape[0]):
        
        content_NAME = results.iloc[i]['content']

        for q in query_path_list:
            # if content_NAME in q:
            if content_NAME == q.split('/')[-1].replace('query_','').replace('.csv',''):
                paht_name = q
                break
        
        large_clf, middel_clf = paht_name.split("query_output/")[1].split('/')[0], paht_name.split("query_output/")[1].split('/')[1]
        results['large'][i] = large_clf
        results['middle'][i] = middel_clf
    return results

In [35]:
def result_to_csv(data, name, line_width, process):
    results = make_results(data, line_width)
    results.reset_index(drop=True)
    if process:
        results['content'] = results['content'].map(lambda x: x[:-2])
        results = analysis_df(results)
    else:
        results = analysis_df(results)
    path = "/DPR/finetuning/facts_net/0104_updated/Error_adjusted_results/"
    results.to_csv(path + f'0104_{name}.csv')
    return results

In [36]:
results = make_results(random, 6)
results.reset_index(drop=True)

results['large'] = 0 
results['middle'] = 0

query_path_list[0].split("query_output/")[1]

for i in range(results.shape[0]):
    
    content_NAME = results.iloc[i]['content']

    for q in query_path_list:
        if content_NAME == q.split('/')[-1].replace('query_','').replace('.csv',''):
            paht_name = q
            break
    
    large_clf, middel_clf = paht_name.split("query_output/")[1].split('/')[0], paht_name.split("query_output/")[1].split('/')[1]
    results['large'][i] = large_clf
    results['middle'][i] = middel_clf

/tmp/ipykernel_3613/227964750.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['large'][i] = large_clf
/tmp/ipykernel_3613/227964750.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['middle'][i] = middel_clf


In [37]:
DRAFT_ss_10000 = result_to_csv(DRAFT_ss_10000, "DRAFT_ss_10000", 6, False)
DRAFT_ss_500 = result_to_csv(DRAFT_ss_500, "DRAFT_ss_500", 6, False)
# DRAFT_ss_50 = result_to_csv(DRAFT_ss_500, "DRAFT_ss_50", False)

gpt3_ada_zero_shot = result_to_csv(gpt3_ada_zero_shot, "gpt3_ada_zero_shot", 8, False)
gpt3_ada_one_shot = result_to_csv(gpt3_ada_one_shot, "gpt3_ada_one_shot", 8, True)
gpt3_ada_three_shot = result_to_csv(gpt3_ada_three_shot, "gpt3_ada_three_shot", 8, True)
gpt3_ada_five_shot = result_to_csv(gpt3_ada_five_shot, "gpt3_ada_five_shot", 8, True)

gpt3_davinci_zero_shot = result_to_csv(gpt3_davinci_zero_shot, "gpt3_davinci_zero_shot", 8, False)
gpt3_davinci_one_shot = result_to_csv(gpt3_davinci_one_shot, "gpt3_davinci_one_shot", 8, True)
gpt3_davinci_three_shot = result_to_csv(gpt3_davinci_three_shot, "gpt3_davinci_three_shot", 8, True)
gpt3_davinci_five_shot = result_to_csv(gpt3_davinci_five_shot, "gpt3_davinci_five_shot",8, True)

random = result_to_csv(random, "random", 6, False)
only_query = result_to_csv(only_query, "only_query", 6, False)
tok_freq = result_to_csv(tok_freq, "tok_freq", 6, False)

import numpy as np

large_cat = np.unique(DRAFT_ss_500['large'])
middle_cat = np.unique(DRAFT_ss_500['middle'])

/tmp/ipykernel_3613/3965591198.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['large'][i] = large_clf
/tmp/ipykernel_3613/3965591198.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['middle'][i] = middel_clf
/tmp/ipykernel_3613/3965591198.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['large'][i] = large_clf
/tmp/ipykernel_3613/3965591198.py:50: SettingWithCopyWarning: 
A value is trying to be se

In [38]:
def results_analysis(categories,category,total_analysis):
    log_ = pd.DataFrame(columns=['based-type','category','f1','acc','prec','rec'])

    for base_type in ['DRAFT_ss_10000', "DRAFT_ss_500", 'gpt3_davinci_zero_shot', 'gpt3_davinci_five_shot', "gpt3_davinci_three_shot", "gpt3_davinci_one_shot", "gpt3_ada_zero_shot", "gpt3_ada_one_shot", "gpt3_ada_three_shot", "gpt3_ada_five_shot", 'random', 'only_query', 'tok_freq']:
        
        if base_type == "DRAFT_ss_10000":
            Results = DRAFT_ss_10000

        elif base_type == "DRAFT_ss_500":
            Results = DRAFT_ss_500


        elif base_type == 'gpt3_davinci_zero_shot':
            # continue
            # davinci zero-shot
            Results = gpt3_davinci_zero_shot

        elif base_type == 'gpt3_davinci_one_shot':
            Results = gpt3_davinci_one_shot

        elif base_type == "gpt3_davinci_three_shot":
            # continue
            Results = gpt3_davinci_three_shot
            
        elif base_type == 'gpt3_davinci_five_shot':
            # continue
            Results = gpt3_davinci_five_shot


        elif base_type == 'gpt3_ada_zero_shot':
            Results = gpt3_ada_zero_shot

        elif base_type == 'gpt3_ada_one_shot':
            Results = gpt3_ada_one_shot

        elif base_type == 'gpt3_ada_three_shot':
            Results = gpt3_ada_three_shot

        elif base_type == 'gpt3_ada_five_shot':
            Results = gpt3_ada_five_shot
            
        elif base_type == 'random':
            Results = random

        elif base_type == 'only_query':
            Results = only_query
            
        elif base_type == 'tok_freq':
            Results = tok_freq


        if total_analysis:
            f1 = round((Results['f1'].sum()) / Results.shape[0],4)
            acc = round((Results['acc'].sum()) / Results.shape[0],4)
            prec = round((Results['prec'].sum()) / Results.shape[0],4)
            rec = round((Results['rec'].sum()) / Results.shape[0],4)
            
            items_ = pd.DataFrame([base_type,'Total',f1,acc,prec, rec])
            items_ = items_.T
            items_.columns = ['based-type','category','f1','acc','prec','rec']
            log_ = pd.concat([log_,items_],axis=0)

        else:
            for lc in categories:

                tmp_df = Results[Results[str(category)]==lc]
                f1 = round((tmp_df['f1'].sum()) / tmp_df.shape[0],4)
                acc = round((tmp_df['acc'].sum()) / tmp_df.shape[0],4)
                prec = round((tmp_df['prec'].sum()) / tmp_df.shape[0],4)
                rec = round((tmp_df['rec'].sum()) / tmp_df.shape[0],4)
                
                items_ = pd.DataFrame([base_type,lc,f1,acc,prec, rec])
                items_ = items_.T
                items_.columns = ['based-type','category','f1','acc','prec','rec']
                log_ = pd.concat([log_,items_],axis=0)
                
    return log_

In [39]:
total_results = results_analysis(None,'Total',True)

In [40]:
large_results = results_analysis(large_cat,'large',False)

In [41]:
middle_results = results_analysis(middle_cat,'middle',False)

In [42]:
path = "/DPR/finetuning/facts_net/0104_updated/Error_adjusted_results/"
total_results.to_csv(path + '0104_total_analysis.csv')
large_results.to_csv(path + '0104_large_analysis.csv')
middle_results.to_csv(path + '0104_middle_analysis.csv')